In [2]:
import pandas as pd 
import numpy as np 
import os 
from pathlib import Path 

In [3]:

stats_summary = pd.read_csv('../../data/preprocessing/womens_summary_season_data.csv')
stats_summary

,Season,TeamName,TeamID,Win_Percentage,Points_Per_Game,FG_Percentage,Threes_Per_Game,Three_Point_Percentage,Free_Throws_Per_Game,Free_Throw_Percentage,Offensive_Rebound_Rate,Defensive_Rebound_Rate,Turnovers_Per_Game,Opp_FG_Percentage,Opp_Three_Point_Percentage,Opp_Free_Throws_Per_Game,Opp_Turnovers_Per_Game,Win_pct_last_10_games
0,2010,Air Force,3102.0,0.035714,51.821429,36.021505,16.071429,28.444444,13.214286,67.837838,34.316618,60.375276,17.714286,47.878404,37.630662,14.892857,14.392857,0.0
1,2010,Akron,3103.0,0.566667,62.233333,40.229192,12.466667,31.550802,19.600000,70.578231,39.664804,63.351254,18.266667,38.144330,31.868132,18.833333,18.700000,0.6
2,2010,Alabama,3104.0,0.379310,64.137931,39.811216,14.724138,29.508197,16.793103,61.601643,35.353535,63.567202,19.172414,39.736239,30.897704,21.965517,19.241379,0.3
3,2010,Alabama A&M,3105.0,0.518519,60.777778,40.058267,9.888889,30.711610,24.333333,69.863014,39.088983,60.614793,25.037037,39.600773,29.275362,23.518519,20.740741,0.6
4,2010,Alabama St,3106.0,0.413793,55.137931,34.808068,9.586207,28.057554,24.000000,64.798851,39.048474,66.666667,20.103448,35.306122,31.486880,26.655172,18.310345,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5597,2025,Stonehill,3476.0,0.478261,63.608696,40.801187,21.304348,33.061224,10.956522,79.761905,29.131653,72.569907,16.391304,42.639594,33.260870,14.347826,13.260870,0.7
5598,2025,East Texas A&M,3477.0,0.181818,61.954545,37.602996,21.409091,31.634820,13.045455,73.170732,20.963542,67.344173,15.181818,43.610108,31.818182,17.818182,13.818182,0.2
5599,2025,Le Moyne,3478.0,0.166667,52.250000,34.234234,20.916667,30.278884,14.666667,75.568182,22.207268,69.708995,17.208333,44.246575,34.804754,12.333333,11.791667,0.4
5600,2025,Mercyhurst,3479.0,0.285714,62.380952,40.594059,21.000000,31.972789,17.428571,72.950820,17.258065,72.439759,17.714286,44.124700,29.261364,18.000000,15.095238,0.5


In [ ]:
## Add team metrics to regular season games to create dataset for supervised ML model
pd.set_option("display.max_columns",None)

stats_summary = pd.read_csv('../../data/preprocessing/womens_summary_season_data.csv')
womens_reg_season_data = pd.read_csv('../../data/WRegularSeasonCompactResults.csv')
womens_reg_season_data = womens_reg_season_data[womens_reg_season_data['Season'] >= 2010]


# Duplicate data to there is a record for each losing and winning team 
womens_reg_season_data['Team1'] = womens_reg_season_data['WTeamID']
womens_reg_season_data['Team2'] = womens_reg_season_data['LTeamID']
womens_reg_season_data['Team1_Wins'] = 1 

flipped = womens_reg_season_data.copy() 
flipped['Team1'], flipped['Team2'] = flipped['Team2'], flipped['Team1'] 
flipped['Team1_Wins'] = 0 

games = pd.concat([womens_reg_season_data, flipped])
games.drop(columns=['WTeamID','WScore','LTeamID','LScore','WLoc','NumOT'], inplace=True)

# Merge games dataset with teams stats 
full_games = games.merge(stats_summary, left_on=['Season','Team1'], right_on=['Season', 'TeamID'], how='left')
full_games = full_games.rename(columns={col: col + "_1" for col in stats_summary.columns if col not in ["Season", "TeamID"]})

full_games = full_games.merge(stats_summary, left_on=['Season','Team2'], right_on=['Season', 'TeamID'], how='left')
full_games = full_games.rename(columns={col: col + "_2" for col in stats_summary.columns if col not in ["Season", "TeamID"]})
full_games.drop(columns=['TeamID_x', 'TeamName_1', 'TeamID_y', 'TeamName_2'], inplace=True)




,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Team1,Team2,Team1_Wins
55101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
135943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
## Add team metrics to regular season games to create dataset for supervised ML model
pd.set_option("display.max_columns",None)

womens_tourney_data = pd.read_csv('../../data/WNCAATourneyCompactResults.csv')
womens_tourney_data = womens_tourney_data[womens_tourney_data['Season'] >= 2010]

womens_tourney_seeds = pd.read_csv("../../data/WNCAATourneySeeds.csv")
womens_tourney_seeds['Seed'] = womens_tourney_seeds['Seed'].str[1:].str.rstrip('ab')

# Duplicate data to there is a record for each losing and winning team 
womens_tourney_data['Team1'] = womens_tourney_data['WTeamID']
womens_tourney_data['Team2'] = womens_tourney_data['LTeamID']
womens_tourney_data['Team1_Wins'] = 1 

flipped = womens_tourney_data.copy() 
flipped['Team1'], flipped['Team2'] = flipped['Team2'], flipped['Team1'] 
flipped['Team1_Wins'] = 0 

games = pd.concat([womens_tourney_data, flipped])
games.drop(columns=['WTeamID','WScore','LTeamID','LScore','WLoc','NumOT'], inplace=True)

# Merge games dataset with teams stats 
tourney_games = games.merge(stats_summary, left_on=['Season','Team1'], right_on=['Season', 'TeamID'], how='left')
tourney_games = tourney_games.rename(columns={col: col + "_1" for col in stats_summary.columns if col not in ["Season", "TeamID"]})

tourney_games = tourney_games.merge(stats_summary, left_on=['Season','Team2'], right_on=['Season', 'TeamID'], how='left')
tourney_games = tourney_games.rename(columns={col: col + "_2" for col in stats_summary.columns if col not in ["Season", "TeamID"]})
tourney_games.drop(columns=['TeamID_x', 'TeamName_1', 'TeamID_y', 'TeamName_2'], inplace=True)

tourney_games = tourney_games.merge(womens_tourney_seeds, how='left', left_on=['Season','Team1'], right_on=['Season', 'TeamID'])
tourney_games = tourney_games.rename(columns={'Seed':'Seed_1'})
tourney_games = tourney_games.merge(womens_tourney_seeds, how='left', left_on=['Season','Team2'], right_on=['Season', 'TeamID'])
tourney_games = tourney_games.rename(columns={'Seed':'Seed_2'})
tourney_games = tourney_games.drop(columns=['TeamID_x', 'TeamID_y'])
tourney_games


,Season,DayNum,Team1,Team2,Team1_Wins,Win_Percentage_1,Points_Per_Game_1,FG_Percentage_1,Threes_Per_Game_1,Three_Point_Percentage_1,Free_Throws_Per_Game_1,Free_Throw_Percentage_1,Offensive_Rebound_Rate_1,Defensive_Rebound_Rate_1,Turnovers_Per_Game_1,Opp_FG_Percentage_1,Opp_Three_Point_Percentage_1,Opp_Free_Throws_Per_Game_1,Opp_Turnovers_Per_Game_1,Win_pct_last_10_games_1,Win_Percentage_2,Points_Per_Game_2,FG_Percentage_2,Threes_Per_Game_2,Three_Point_Percentage_2,Free_Throws_Per_Game_2,Free_Throw_Percentage_2,Offensive_Rebound_Rate_2,Defensive_Rebound_Rate_2,Turnovers_Per_Game_2,Opp_FG_Percentage_2,Opp_Three_Point_Percentage_2,Opp_Free_Throws_Per_Game_2,Opp_Turnovers_Per_Game_2,Win_pct_last_10_games_2,Seed_1,Seed_2
0,2010,138,3124,3201,1,0.718750,72.093750,46.311703,8.656250,28.519856,24.000000,72.656250,35.878565,71.000000,16.875000,33.732535,27.742947,13.187500,15.718750,0.6,0.818182,73.454545,41.978346,24.848485,35.487805,18.242424,70.930233,35.865309,66.748971,15.636364,40.459647,29.978118,16.303030,19.878788,0.9,04,13
1,2010,138,3173,3395,1,0.807692,70.769231,42.062500,17.461538,31.277533,19.346154,69.980119,38.261738,69.923372,16.961538,36.762779,32.191781,16.576923,18.346154,0.8,0.733333,72.300000,41.488223,19.200000,35.763889,18.400000,74.818841,34.763948,63.472563,16.033333,36.571748,28.429752,16.300000,20.666667,0.7,08,09
2,2010,138,3181,3214,1,0.843750,71.593750,42.107843,14.468750,31.533477,20.156250,66.201550,44.645669,67.186218,17.906250,35.160350,32.015066,18.125000,23.218750,0.9,0.633333,62.833333,36.938436,16.100000,30.434783,18.933333,71.478873,36.415247,64.211520,15.533333,38.050734,27.741935,22.000000,22.133333,0.9,02,15
3,2010,138,3199,3256,1,0.833333,74.466667,44.594595,16.100000,37.888199,21.800000,71.406728,40.487348,67.049180,19.033333,35.814214,28.752260,17.466667,19.200000,0.8,0.741935,74.258065,43.446852,12.451613,31.606218,23.354839,68.508287,37.715694,68.448404,17.096774,37.311178,27.643785,18.000000,17.451613,0.8,03,14
4,2010,138,3207,3265,1,0.800000,68.033333,39.546460,20.066667,31.561462,19.533333,71.843003,40.775862,61.668427,16.300000,40.381992,28.672566,18.766667,24.600000,0.7,0.787879,68.181818,42.056583,19.030303,34.394904,18.484848,80.000000,31.037567,64.727815,13.484848,34.910026,29.645390,14.909091,15.727273,0.8,05,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2024,147,3425,3163,0,0.838710,74.677419,42.568617,21.225806,35.562310,18.838710,74.828767,31.795878,75.794872,11.806452,38.790233,30.671506,15.387097,14.548387,0.9,0.848485,79.878788,49.557522,19.787879,35.834609,15.484848,75.538160,29.257642,77.150786,12.363636,36.231130,28.370787,12.242424,15.969697,0.9,01,03
1784,2024,147,3261,3234,0,0.848485,86.696970,46.712018,13.212121,32.110092,26.727273,74.943311,44.225604,75.893599,15.212121,36.731707,29.613734,15.969697,18.939394,0.9,0.878788,92.848485,50.322284,29.606061,38.178096,19.575758,78.173375,29.596853,76.206051,13.424242,40.101056,31.055156,14.969697,14.424242,0.8,03,01
1785,2024,151,3163,3234,0,0.848485,79.878788,49.557522,19.787879,35.834609,15.484848,75.538160,29.257642,77.150786,12.363636,36.231130,28.370787,12.242424,15.969697,0.9,0.878788,92.848485,50.322284,29.606061,38.178096,19.575758,78.173375,29.596853,76.206051,13.424242,40.101056,31.055156,14.969697,14.424242,0.8,03,01
1786,2024,151,3301,3376,0,0.818182,73.818182,42.919075,18.636364,32.357724,18.696970,73.743922,28.028933,77.186010,12.333333,35.453695,29.178082,10.060606,12.696970,0.7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,03,01


In [22]:
output_dir = Path("../..") / "data" / "modeling"
output_dir.mkdir(parents=True, exist_ok =True)
season_output_path = output_dir / "womens_reg_season_ml.csv"
tourney_output_path = output_dir / "womens_tourney_ml.csv"
full_games.to_csv(season_output_path, index=False)
tourney_games.to_csv(tourney_output_path, index=False)